In [1]:
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage

In [31]:
# import Librairies
import pandas as pd
import numpy as  np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO
import os

In [3]:
# Function

import os
import boto3
from azure.storage.blob import BlobServiceClient
from google.cloud import storage
import pandas as pd
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

# Google Cloud Functions
def google_upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

def google_download_blob(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

# AWS Functions
def aws_upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(file_name, bucket, object_name)
    print(f"Uploaded {file_name} to S3 bucket {bucket}.")

def aws_download_file(bucket, object_name, file_name):
    s3_client = boto3.client('s3')
    s3_client.download_file(bucket, object_name, file_name)
    print(f"Downloaded {object_name} from S3 bucket {bucket}.")

In [15]:
NYPD_COMPLAINTS = pd.read_csv("NYPD_Complaint_Map__Year_to_Date_.csv")

NYPD_COMPLAINTS.head(20)

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,JURIS_DESC,...,PARKS_NM,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,Lat_Lon,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,269235387,42.0,BRONX,6/3/2023,1:00:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),366.0,"ROBBERY,BICYCLE",STREET,6/3/2023,POINT (-73.89979939 40.82405906),1011982.0,239520.0,40.824059,-73.899799
1,262102261,43.0,BRONX,1/18/2023,14:00:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),725.0,"FORGERY,M.V. REGISTRATION",STREET,1/18/2023,POINT (-73.87212505 40.83585867),1019635.0,243829.0,40.835859,-73.872125
2,267972239,43.0,BRONX,5/7/2023,0:15:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),105.0,STRANGULATION 1ST,STREET,5/9/2023,POINT (-73.8634 40.8315),1022051.0,242244.0,40.831500,-73.863400
3,263141402H1,NaN,BROOKLYN,2/6/2023,10:16:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),NaN,(null),MULTI DWELL - APT BUILD,2/6/2023,NaN,NaN,NaN,NaN,NaN
4,269307857H1,NaN,BROOKLYN,6/4/2023,0:20:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),NaN,(null),(null),6/4/2023,NaN,NaN,NaN,NaN,NaN
5,277525074H3,NaN,QUEENS,11/14/2023,7:05:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),NaN,(null),PVT HOUSE,11/14/2023,NaN,NaN,NaN,NaN,NaN
6,277525074H1,NaN,QUEENS,11/14/2023,7:05:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),NaN,(null),PVT HOUSE,11/14/2023,NaN,NaN,NaN,NaN,NaN
7,271421229H1,NaN,BROOKLYN,7/5/2020,21:45:00,NaN,(null),COMPLETED,NOSTRAND,N.Y. HOUSING POLICE,...,(null),NaN,(null),MULTI DWELL - PUBLIC HOUS,7/5/2023,NaN,NaN,NaN,NaN,NaN
8,270206142H1,NaN,MANHATTAN,6/21/2023,16:25:00,NaN,(null),COMPLETED,(null),N.Y. POLICE DEPT,...,(null),NaN,(null),(null),6/21/2023,NaN,NaN,NaN,NaN,NaN
9,278972931,123.0,STATEN ISLAND,12/14/2023,2:50:00,12/14/2023,2:52:00,COMPLETED,(null),N.Y. POLICE DEPT,...,(null),511.0,"CONTROLLED SUBSTANCE, POSSESSI",STREET,12/14/2023,POINT (-74.18043031626128 40.54653060020265),934105.0,138444.0,40.546531,-74.180430


In [16]:
print(NYPD_COMPLAINTS.columns)

Index(['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM',
       'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT',
       'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC',
       'PARKS_NM', 'PD_CD', 'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT', 'Lat_Lon',
       'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude'],
      dtype='object')


In [17]:
NYPD_COMPLAINTS.shape

(555117, 24)

In [18]:
NYPD_COMPLAINTS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555117 entries, 0 to 555116
Data columns (total 24 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CMPLNT_NUM         555117 non-null  object 
 1   ADDR_PCT_CD        555047 non-null  float64
 2   BORO_NM            555117 non-null  object 
 3   CMPLNT_FR_DT       555117 non-null  object 
 4   CMPLNT_FR_TM       555117 non-null  object 
 5   CMPLNT_TO_DT       520379 non-null  object 
 6   CMPLNT_TO_TM       555117 non-null  object 
 7   CRM_ATPT_CPTD_CD   555117 non-null  object 
 8   HADEVELOPT         555117 non-null  object 
 9   JURIS_DESC         555117 non-null  object 
 10  KY_CD              555117 non-null  int64  
 11  LAW_CAT_CD         555117 non-null  object 
 12  LOC_OF_OCCUR_DESC  555117 non-null  object 
 13  OFNS_DESC          555117 non-null  object 
 14  PARKS_NM           555117 non-null  object 
 15  PD_CD              554731 non-null  float64
 16  PD

In [19]:
NYPD_COMPLAINTS_CLEAN = NYPD_COMPLAINTS.copy()

In [20]:
NYPD_COMPLAINTS_CLEAN = NYPD_COMPLAINTS.drop(columns=  ['HADEVELOPT', 'PARKS_NM', 'Lat_Lon' ])
NYPD_COMPLAINTS_CLEAN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555117 entries, 0 to 555116
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CMPLNT_NUM         555117 non-null  object 
 1   ADDR_PCT_CD        555047 non-null  float64
 2   BORO_NM            555117 non-null  object 
 3   CMPLNT_FR_DT       555117 non-null  object 
 4   CMPLNT_FR_TM       555117 non-null  object 
 5   CMPLNT_TO_DT       520379 non-null  object 
 6   CMPLNT_TO_TM       555117 non-null  object 
 7   CRM_ATPT_CPTD_CD   555117 non-null  object 
 8   JURIS_DESC         555117 non-null  object 
 9   KY_CD              555117 non-null  int64  
 10  LAW_CAT_CD         555117 non-null  object 
 11  LOC_OF_OCCUR_DESC  555117 non-null  object 
 12  OFNS_DESC          555117 non-null  object 
 13  PD_CD              554731 non-null  float64
 14  PD_DESC            555117 non-null  object 
 15  PREM_TYP_DESC      555117 non-null  object 
 16  RP

In [21]:
#Fill in null values for the column "ADDR_PCT_CD"
NYPD_COMPLAINTS_CLEAN["ADDR_PCT_CD"] = NYPD_COMPLAINTS_CLEAN["ADDR_PCT_CD"].fillna("00")
NYPD_COMPLAINTS_CLEAN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555117 entries, 0 to 555116
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CMPLNT_NUM         555117 non-null  object 
 1   ADDR_PCT_CD        555117 non-null  object 
 2   BORO_NM            555117 non-null  object 
 3   CMPLNT_FR_DT       555117 non-null  object 
 4   CMPLNT_FR_TM       555117 non-null  object 
 5   CMPLNT_TO_DT       520379 non-null  object 
 6   CMPLNT_TO_TM       555117 non-null  object 
 7   CRM_ATPT_CPTD_CD   555117 non-null  object 
 8   JURIS_DESC         555117 non-null  object 
 9   KY_CD              555117 non-null  int64  
 10  LAW_CAT_CD         555117 non-null  object 
 11  LOC_OF_OCCUR_DESC  555117 non-null  object 
 12  OFNS_DESC          555117 non-null  object 
 13  PD_CD              554731 non-null  float64
 14  PD_DESC            555117 non-null  object 
 15  PREM_TYP_DESC      555117 non-null  object 
 16  RP

In [ ]:
#fill in other Null Values for respective columns

In [22]:

NYPD_COMPLAINTS_CLEAN["CMPLNT_TO_DT"] = NYPD_COMPLAINTS_CLEAN["CMPLNT_TO_DT"].fillna("12/31/2023")
NYPD_COMPLAINTS_CLEAN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555117 entries, 0 to 555116
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CMPLNT_NUM         555117 non-null  object 
 1   ADDR_PCT_CD        555117 non-null  object 
 2   BORO_NM            555117 non-null  object 
 3   CMPLNT_FR_DT       555117 non-null  object 
 4   CMPLNT_FR_TM       555117 non-null  object 
 5   CMPLNT_TO_DT       555117 non-null  object 
 6   CMPLNT_TO_TM       555117 non-null  object 
 7   CRM_ATPT_CPTD_CD   555117 non-null  object 
 8   JURIS_DESC         555117 non-null  object 
 9   KY_CD              555117 non-null  int64  
 10  LAW_CAT_CD         555117 non-null  object 
 11  LOC_OF_OCCUR_DESC  555117 non-null  object 
 12  OFNS_DESC          555117 non-null  object 
 13  PD_CD              554731 non-null  float64
 14  PD_DESC            555117 non-null  object 
 15  PREM_TYP_DESC      555117 non-null  object 
 16  RP

In [23]:
NYPD_COMPLAINTS_CLEAN["PD_CD"] = NYPD_COMPLAINTS_CLEAN["PD_CD"].fillna("000")
NYPD_COMPLAINTS_CLEAN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555117 entries, 0 to 555116
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CMPLNT_NUM         555117 non-null  object 
 1   ADDR_PCT_CD        555117 non-null  object 
 2   BORO_NM            555117 non-null  object 
 3   CMPLNT_FR_DT       555117 non-null  object 
 4   CMPLNT_FR_TM       555117 non-null  object 
 5   CMPLNT_TO_DT       555117 non-null  object 
 6   CMPLNT_TO_TM       555117 non-null  object 
 7   CRM_ATPT_CPTD_CD   555117 non-null  object 
 8   JURIS_DESC         555117 non-null  object 
 9   KY_CD              555117 non-null  int64  
 10  LAW_CAT_CD         555117 non-null  object 
 11  LOC_OF_OCCUR_DESC  555117 non-null  object 
 12  OFNS_DESC          555117 non-null  object 
 13  PD_CD              555117 non-null  object 
 14  PD_DESC            555117 non-null  object 
 15  PREM_TYP_DESC      555117 non-null  object 
 16  RP

In [24]:
# Fill in "0" for the remaining columns all at once

columns_to_fill = ["X_COORD_CD", "Y_COORD_CD", "Latitude", "Longitude"]
NYPD_COMPLAINTS_CLEAN[columns_to_fill] = NYPD_COMPLAINTS_CLEAN[columns_to_fill].fillna("0")
NYPD_COMPLAINTS_CLEAN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555117 entries, 0 to 555116
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   CMPLNT_NUM         555117 non-null  object
 1   ADDR_PCT_CD        555117 non-null  object
 2   BORO_NM            555117 non-null  object
 3   CMPLNT_FR_DT       555117 non-null  object
 4   CMPLNT_FR_TM       555117 non-null  object
 5   CMPLNT_TO_DT       555117 non-null  object
 6   CMPLNT_TO_TM       555117 non-null  object
 7   CRM_ATPT_CPTD_CD   555117 non-null  object
 8   JURIS_DESC         555117 non-null  object
 9   KY_CD              555117 non-null  int64 
 10  LAW_CAT_CD         555117 non-null  object
 11  LOC_OF_OCCUR_DESC  555117 non-null  object
 12  OFNS_DESC          555117 non-null  object
 13  PD_CD              555117 non-null  object
 14  PD_DESC            555117 non-null  object
 15  PREM_TYP_DESC      555117 non-null  object
 16  RPT_DT             5

In [25]:
NYPD_COMPLAINTS_CLEAN

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURIS_DESC,KY_CD,...,LOC_OF_OCCUR_DESC,OFNS_DESC,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,269235387,42.0,BRONX,6/3/2023,1:00:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,105,...,(null),ROBBERY,366.0,"ROBBERY,BICYCLE",STREET,6/3/2023,1011982.0,239520.0,40.824059,-73.899799
1,262102261,43.0,BRONX,1/18/2023,14:00:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,113,...,(null),FORGERY,725.0,"FORGERY,M.V. REGISTRATION",STREET,1/18/2023,1019635.0,243829.0,40.835859,-73.872125
2,267972239,43.0,BRONX,5/7/2023,0:15:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,106,...,FRONT OF,FELONY ASSAULT,105.0,STRANGULATION 1ST,STREET,5/9/2023,1022051.0,242244.0,40.8315,-73.8634
3,263141402H1,00,BROOKLYN,2/6/2023,10:16:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,101,...,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,000,(null),MULTI DWELL - APT BUILD,2/6/2023,0,0,0,0
4,269307857H1,00,BROOKLYN,6/4/2023,0:20:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,101,...,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,000,(null),(null),6/4/2023,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555112,278932501,105.0,QUEENS,12/12/2023,21:00:00,12/12/2023,21:12:00,COMPLETED,N.Y. POLICE DEPT,361,...,INSIDE,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,RESIDENCE - APT. HOUSE,12/12/2023,1057934.0,198975.0,40.712511,-73.73421
555113,274543137,105.0,QUEENS,9/16/2023,12:20:00,9/16/2023,12:25:00,COMPLETED,N.Y. POLICE DEPT,344,...,INSIDE,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,RESIDENCE-HOUSE,9/16/2023,1064159.0,204907.0,40.728739,-73.711686
555114,272734727,105.0,QUEENS,7/15/2023,10:50:00,7/15/2023,11:00:00,COMPLETED,N.Y. POLICE DEPT,112,...,INSIDE,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",(null),8/10/2023,1062587.0,206274.0,40.732507,-73.71734
555115,270806588,105.0,QUEENS,7/4/2023,21:00:00,7/4/2023,21:15:00,COMPLETED,N.Y. POLICE DEPT,126,...,(null),MISCELLANEOUS PENAL LAW,117.0,RECKLESS ENDANGERMENT 1,STREET,7/4/2023,1059585.0,198510.0,40.71122,-73.728262


In [27]:
# Replace "(null)" with "NOT REPORTED" for specified columns
columns_to_replace = ["LOC_OF_OCCUR_DESC", "PD_DESC", "PREM_TYP_DESC"]
NYPD_COMPLAINTS_CLEAN[columns_to_replace] = NYPD_COMPLAINTS_CLEAN[columns_to_replace].replace("(null)", "NOT REPORTED")

In [28]:
NYPD_COMPLAINTS_CLEAN

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURIS_DESC,KY_CD,...,LOC_OF_OCCUR_DESC,OFNS_DESC,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,269235387,42.0,BRONX,6/3/2023,1:00:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,105,...,NOT REPORTED,ROBBERY,366.0,"ROBBERY,BICYCLE",STREET,6/3/2023,1011982.0,239520.0,40.824059,-73.899799
1,262102261,43.0,BRONX,1/18/2023,14:00:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,113,...,NOT REPORTED,FORGERY,725.0,"FORGERY,M.V. REGISTRATION",STREET,1/18/2023,1019635.0,243829.0,40.835859,-73.872125
2,267972239,43.0,BRONX,5/7/2023,0:15:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,106,...,FRONT OF,FELONY ASSAULT,105.0,STRANGULATION 1ST,STREET,5/9/2023,1022051.0,242244.0,40.8315,-73.8634
3,263141402H1,00,BROOKLYN,2/6/2023,10:16:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,101,...,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,000,NOT REPORTED,MULTI DWELL - APT BUILD,2/6/2023,0,0,0,0
4,269307857H1,00,BROOKLYN,6/4/2023,0:20:00,12/31/2023,(null),COMPLETED,N.Y. POLICE DEPT,101,...,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,000,NOT REPORTED,NOT REPORTED,6/4/2023,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555112,278932501,105.0,QUEENS,12/12/2023,21:00:00,12/12/2023,21:12:00,COMPLETED,N.Y. POLICE DEPT,361,...,INSIDE,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,RESIDENCE - APT. HOUSE,12/12/2023,1057934.0,198975.0,40.712511,-73.73421
555113,274543137,105.0,QUEENS,9/16/2023,12:20:00,9/16/2023,12:25:00,COMPLETED,N.Y. POLICE DEPT,344,...,INSIDE,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,RESIDENCE-HOUSE,9/16/2023,1064159.0,204907.0,40.728739,-73.711686
555114,272734727,105.0,QUEENS,7/15/2023,10:50:00,7/15/2023,11:00:00,COMPLETED,N.Y. POLICE DEPT,112,...,INSIDE,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",NOT REPORTED,8/10/2023,1062587.0,206274.0,40.732507,-73.71734
555115,270806588,105.0,QUEENS,7/4/2023,21:00:00,7/4/2023,21:15:00,COMPLETED,N.Y. POLICE DEPT,126,...,NOT REPORTED,MISCELLANEOUS PENAL LAW,117.0,RECKLESS ENDANGERMENT 1,STREET,7/4/2023,1059585.0,198510.0,40.71122,-73.728262


In [29]:
NYPD_COMPLAINTS_CLEAN["CMPLNT_TO_TM"] = NYPD_COMPLAINTS_CLEAN["CMPLNT_TO_TM"].replace("(null)", "23:59:59")

In [30]:
NYPD_COMPLAINTS_CLEAN

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURIS_DESC,KY_CD,...,LOC_OF_OCCUR_DESC,OFNS_DESC,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,269235387,42.0,BRONX,6/3/2023,1:00:00,12/31/2023,23:59:59,COMPLETED,N.Y. POLICE DEPT,105,...,NOT REPORTED,ROBBERY,366.0,"ROBBERY,BICYCLE",STREET,6/3/2023,1011982.0,239520.0,40.824059,-73.899799
1,262102261,43.0,BRONX,1/18/2023,14:00:00,12/31/2023,23:59:59,COMPLETED,N.Y. POLICE DEPT,113,...,NOT REPORTED,FORGERY,725.0,"FORGERY,M.V. REGISTRATION",STREET,1/18/2023,1019635.0,243829.0,40.835859,-73.872125
2,267972239,43.0,BRONX,5/7/2023,0:15:00,12/31/2023,23:59:59,COMPLETED,N.Y. POLICE DEPT,106,...,FRONT OF,FELONY ASSAULT,105.0,STRANGULATION 1ST,STREET,5/9/2023,1022051.0,242244.0,40.8315,-73.8634
3,263141402H1,00,BROOKLYN,2/6/2023,10:16:00,12/31/2023,23:59:59,COMPLETED,N.Y. POLICE DEPT,101,...,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,000,NOT REPORTED,MULTI DWELL - APT BUILD,2/6/2023,0,0,0,0
4,269307857H1,00,BROOKLYN,6/4/2023,0:20:00,12/31/2023,23:59:59,COMPLETED,N.Y. POLICE DEPT,101,...,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,000,NOT REPORTED,NOT REPORTED,6/4/2023,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555112,278932501,105.0,QUEENS,12/12/2023,21:00:00,12/12/2023,21:12:00,COMPLETED,N.Y. POLICE DEPT,361,...,INSIDE,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,RESIDENCE - APT. HOUSE,12/12/2023,1057934.0,198975.0,40.712511,-73.73421
555113,274543137,105.0,QUEENS,9/16/2023,12:20:00,9/16/2023,12:25:00,COMPLETED,N.Y. POLICE DEPT,344,...,INSIDE,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,RESIDENCE-HOUSE,9/16/2023,1064159.0,204907.0,40.728739,-73.711686
555114,272734727,105.0,QUEENS,7/15/2023,10:50:00,7/15/2023,11:00:00,COMPLETED,N.Y. POLICE DEPT,112,...,INSIDE,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",NOT REPORTED,8/10/2023,1062587.0,206274.0,40.732507,-73.71734
555115,270806588,105.0,QUEENS,7/4/2023,21:00:00,7/4/2023,21:15:00,COMPLETED,N.Y. POLICE DEPT,126,...,NOT REPORTED,MISCELLANEOUS PENAL LAW,117.0,RECKLESS ENDANGERMENT 1,STREET,7/4/2023,1059585.0,198510.0,40.71122,-73.728262


In [32]:
# Save the DataFrame to a CSV file on your desktop
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
file_path = os.path.join(desktop_path, "NYPD_COMPLAINTS_CLEAN.csv")
NYPD_COMPLAINTS_CLEAN.to_csv(file_path, index=False)